In [1]:
# 모듈 import

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.utils import get_device

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import wandb


In [14]:
model_name = "wandb_val_test"

In [15]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
#cfg = Config.fromfile('./configs/a_my_config/my_config.py')
cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')
root='../../../../../dataset/'

# dataset config 수정
cfg.data.train.classes = classes
#cfg.data.train.palette = palette

cfg.data.train.img_prefix = root
cfg.data.train.ann_file = root + 'split_train.json' # train json 정보
cfg.data.train.pipeline[2]['img_scale'] = (512,512) # Resize

cfg.data.val.classes = classes
cfg.data.val.img_prefix = root
cfg.data.val.ann_file = root + 'split_val.json' # test json 정보
cfg.data.val.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.samples_per_gpu = 2
cfg.data.workers_per_gpu = 2

cfg.seed = 20
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/'+ model_name

cfg.model.roi_head.bbox_head.num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)
cfg.device = get_device()

cfg.runner.max_epochs = 30

In [16]:
cfg.workflow = [('train', 1)]

In [17]:
cfg.runner.meta = dict(exp_name = "exp_name")

In [18]:
wandb.login()

cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='MyMMDetWandbHook',
         init_kwargs={'project' : 'object-detection',
                      'name' : model_name},
         interval=10,
         log_checkpoint=True,
         log_checkpoint_metadata=True,
         num_eval_images=100
         )
         ]


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [19]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]
#valid_datasets = []

loading annotations into memory...
Done (t=0.33s)
creating index...
index created!


In [9]:
# dataset 확인
datasets[0]


CocoDataset Train dataset with number of images 3906, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 3179  | 1 [Paper]     | 5213  | 2 [Paper pack]  | 707   | 3 [Metal]   | 764   | 4 [Glass]    | 727   |
| 5 [Plastic]       | 2375  | 6 [Styrofoam] | 966   | 7 [Plastic bag] | 4114  | 8 [Battery] | 138   | 9 [Clothing] | 374   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [24]:
cfg.workflow = [('train', 1)]

In [25]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

2024-10-16 15:19:52,248 - mmcv - INFO - initialize ResNet with init_cfg {'type': 'Pretrained', 'checkpoint': 'torchvision://resnet50'}
2024-10-16 15:19:52,250 - mmcv - INFO - load model from: torchvision://resnet50
2024-10-16 15:19:52,251 - mmcv - INFO - load checkpoint from torchvision path: torchvision://resnet50
2024-10-16 15:19:52,349 - mmcv - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

2024-10-16 15:19:52,366 - mmcv - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2024-10-16 15:19:52,394 - mmcv - INFO - initialize RPNHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}
2024-10-16 15:19:52,400 - mmcv - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}}, {'type': 'Xavier', 'distribution': 'uniform', 'override'

In [26]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

2024-10-16 15:19:53,812 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2024-10-16 15:19:53,840 - mmdet - INFO - Start running, host: root@instance-12243, work_dir: /data/ephemeral/home/KSG-git/level2-objectdetection-cv-20/code/baseline/mmdetection/work_dirs/wandb_val_test
2024-10-16 15:19:53,841 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) MyMMDetWandbHook                   
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) MyMMDetWandbHook              

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


learning_rate,▁▂▂▃▃▄▄▅▅▆▇▇████████████████████████████
momentum,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/acc,██████████▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/grad_norm,▁▁▁▁▁▁▁▁▁▁▁▁▁▁█
train/loss,▁▁▁▁▁▁▁▁▁▁▆████▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▃▃▃▃▃▃
train/loss_bbox,▁▁▁▁▁▁▁▁▁▄▄▃▅▂▃▄▃▅▄▃█▅▄▂▃▂▄▅▂▃▂▃▃▂▂▂▂▂▂▂
train/loss_cls,▁▁▁▁▁▁▁▁▁█████▇▇▇▇▇▆▆▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▃▃▃▃
train/loss_rpn_bbox,▁▁▁▁▁▆▅█▅▆▄▄▅▇▇▄▅▅▅▃▃▅▅▃▅▂▅▆▂▂▃▅▄▂▃▂▃▂▂▂
train/loss_rpn_cls,▁▁▁▁▁▁▁▁▁▁▁▄▅█▆▇█▅▅▃▅▅▅▇▄▃▂▄▃▂▅▃▃▃▂▂▂▃▂▃
val/bbox_mAP,▁
val/bbox_mAP_50,▁


wandb: Downloading large artifact val:v0, 152.82MB. 102 files... 
wandb:   102 of 102 files downloaded.  
Done. 0:0:0.8
2024-10-16 15:20:58,872 - mmdet - INFO - Epoch [1][50/1953]	lr: 1.978e-03, eta: 2:36:49, time: 0.161, data_time: 0.047, memory: 1942, loss_rpn_cls: 0.3944, loss_rpn_bbox: 0.0499, loss_cls: 0.5984, acc: 89.0898, loss_bbox: 0.1087, loss: 1.1513, grad_norm: 8.3571
2024-10-16 15:21:04,609 - mmdet - INFO - Epoch [1][100/1953]	lr: 3.976e-03, eta: 2:14:16, time: 0.115, data_time: 0.004, memory: 1942, loss_rpn_cls: 0.1936, loss_rpn_bbox: 0.0488, loss_cls: 0.3170, acc: 94.8105, loss_bbox: 0.1921, loss: 0.7515, grad_norm: 4.7601
2024-10-16 15:21:10,287 - mmdet - INFO - Epoch [1][150/1953]	lr: 5.974e-03, eta: 2:06:18, time: 0.114, data_time: 0.004, memory: 1942, loss_rpn_cls: 0.1685, loss_rpn_bbox: 0.0501, loss_cls: 0.3395, acc: 94.2383, loss_bbox: 0.2198, loss: 0.7779, grad_norm: 3.2772
2024-10-16 15:21:15,979 - mmdet - INFO - Epoch [1][200/1953]	lr: 7.972e-03, eta: 2:02:20, ti

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 977/977, 30.0 task/s, elapsed: 33s, ETA:     0s

...

2024-10-16 16:38:15,989 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.03s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=3.69s).
Accumulating evaluation results...


2024-10-16 16:38:20,912 - mmdet - INFO - 
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.233
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.369
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.243
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.004
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.023
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.282
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.441
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.441
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.441
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.012
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.109
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000

DONE (t=1.08s).
Loading and preparing results...
DONE (t=0.34s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=3.40s).
Accumulating evaluation results...
DONE (t=1.11s).


2024-10-16 16:38:42,284 - mmdet - INFO - Epoch [18][50/1953]	lr: 2.000e-04, eta: 0:47:41, time: 0.156, data_time: 0.048, memory: 1942, loss_rpn_cls: 0.0257, loss_rpn_bbox: 0.0297, loss_cls: 0.2249, acc: 93.0176, loss_bbox: 0.1860, loss: 0.4664, grad_norm: 2.1661
2024-10-16 16:38:47,939 - mmdet - INFO - Epoch [18][100/1953]	lr: 2.000e-04, eta: 0:47:35, time: 0.113, data_time: 0.004, memory: 1942, loss_rpn_cls: 0.0202, loss_rpn_bbox: 0.0250, loss_cls: 0.2134, acc: 93.2656, loss_bbox: 0.1788, loss: 0.4373, grad_norm: 2.1100
2024-10-16 16:38:53,483 - mmdet - INFO - Epoch [18][150/1953]	lr: 2.000e-04, eta: 0:47:30, time: 0.111, data_time: 0.004, memory: 1942, loss_rpn_cls: 0.0186, loss_rpn_bbox: 0.0220, loss_cls: 0.1973, acc: 93.9102, loss_bbox: 0.1706, loss: 0.4085, grad_norm: 2.1658
2024-10-16 16:38:58,969 - mmdet - INFO - Epoch [18][200/1953]	lr: 2.000e-04, eta: 0:47:24, time: 0.110, data_time: 0.004, memory: 1942, loss_rpn_cls: 0.0179, loss_rpn_bbox: 0.0234, loss_cls: 0.2084, acc: 93.22